In [ ]:
%matplotlib inline


# Checkpoint using joblib.Memory and joblib.Parallel

This example illustrates how to cache intermediate computing results using
:class:`joblib.Memory` within :class:`joblib.Parallel`.


## Embed caching within parallel processing

 It is possible to cache a computationally expensive function executed during
 a parallel process. ``costly_compute`` emulates such time consuming function.



In [1]:
import time


def costly_compute(data, column):
    """Emulate a costly function by sleeping and returning a column."""
    time.sleep(2)
    return data[column]

def data_processing_mean(data, column):
    """Compute the mean of a column."""
    return costly_compute(data, column).mean()

Create some data. The random seed is fixed to generate deterministic data
across Python session. Note that this is not necessary for this specific
example since the memory cache is cleared at the end of the session.



In [3]:
import numpy as np
rng = np.random.RandomState(42)
data = rng.randn(int(1e4), 4)
data

array([[ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986],
       [-0.23415337, -0.23413696,  1.57921282,  0.76743473],
       [-0.46947439,  0.54256004, -0.46341769, -0.46572975],
       ...,
       [ 2.71049244,  0.77279139,  1.82925283,  0.99348728],
       [ 0.15851616, -0.59765756, -0.97683992,  1.11965102],
       [ 0.26503405,  0.61650104,  0.8146235 ,  0.35597439]])

It is first possible to make the processing without caching or parallel
processing.



In [4]:
start = time.time()
results = [data_processing_mean(data, col) for col in range(data.shape[1])]
stop = time.time()

print('\nSequential processing')
print('Elapsed time for the entire processing: {:.2f} s'
      .format(stop - start))


Sequential processing
Elapsed time for the entire processing: 8.00 s


``costly_compute`` is expensive to compute and it is used as an intermediate
step in ``data_processing_mean``. Therefore, it is interesting to store the
intermediate results from ``costly_compute`` using :class:`joblib.Memory`.



In [5]:
from joblib import Memory

location = './cachedir'
memory = Memory(location, verbose=0)
costly_compute_cached = memory.cache(costly_compute)

Now, we define ``data_processing_mean_using_cache`` which benefits from the
cache by calling ``costly_compute_cached``



In [6]:
def data_processing_mean_using_cache(data, column):
    """Compute the mean of a column."""
    return costly_compute_cached(data, column).mean()

Then, we execute the same processing in parallel and caching the intermediate
results.



In [7]:
from joblib import Parallel, delayed

start = time.time()
results = Parallel(n_jobs=2)(
    delayed(data_processing_mean_using_cache)(data, col)
    for col in range(data.shape[1]))
stop = time.time()

print('\nFirst round - caching the data')
print('Elapsed time for the entire processing: {:.2f} s'
      .format(stop - start))


First round - caching the data
Elapsed time for the entire processing: 4.99 s


By using 2 workers, the parallel processing gives a x2 speed-up compared to
the sequential case. By executing again the same process, the intermediate
results obtained by calling ``costly_compute_cached`` will be loaded from the
cache instead of executing the function.



In [8]:
start = time.time()
results = Parallel(n_jobs=2)(
    delayed(data_processing_mean_using_cache)(data, col)
    for col in range(data.shape[1]))
stop = time.time()

print('\nSecond round - reloading from the cache')
print('Elapsed time for the entire processing: {:.2f} s'
      .format(stop - start))


Second round - reloading from the cache
Elapsed time for the entire processing: 0.05 s


## Reuse intermediate checkpoints

 Having cached the intermediate results of the ``costly_compute_cached``
 function, they are reusable by calling the function. We define a new
 processing which will take the maximum of the array returned by
 ``costly_compute_cached`` instead of previously the mean.



In [9]:
def data_processing_max_using_cache(data, column):
    """Compute the max of a column."""
    return costly_compute_cached(data, column).max()


start = time.time()
results = Parallel(n_jobs=2)(
    delayed(data_processing_max_using_cache)(data, col)
    for col in range(data.shape[1]))
stop = time.time()

print('\nReusing intermediate checkpoints')
print('Elapsed time for the entire processing: {:.2f} s'
      .format(stop - start))


Reusing intermediate checkpoints
Elapsed time for the entire processing: 0.02 s


The processing time only corresponds to the execution of the ``max``
function. The internal call to ``costly_compute_cached`` is reloading the
results from the cache.



## Clean-up the cache folder



In [10]:
memory.clear(warn=False)